In [18]:
#setup data
import csv
import pandas as pd
import numpy as np

#define risk free rate(rfr) as the three month treasure yield (DGS3MO)
rfr = np.array((5244,1))
#define term spread as 10 year treasure interest(DGS10) minus 1 year treasure interest(DGS1)
ts = np.array((5244,1))
#define default spread as the yield gap between AAA(DAAA) and BBB(DBAA) grade bonds
ds = np.array((5244,1))
ones = np.ones((5243,1))[:,0]

data1 = pd.read_csv('spread.csv')
data2 = pd.read_csv('StockMarket.csv')
data = pd.merge(data1, data2, on='Date')
rfr = data['DGS3MO']
ds = data['DGS10'] - data['DGS1']
ts = data['DBAA'] - data['DAAA']
# print(rfr.shape)
# print(ds.shape)
# print(ts.shape)
# print(ones.shape)
# A = np.stack((rfr, ds, ts, ones), axis = 1)

rfr2=np.zeros((5243,))
ds2=np.zeros((5243,))
ts2=np.zeros((5243,))
for i in range(5243):
    rfr2[i] = rfr[i+1]-rfr[i]
    ds2[i] = ds[i+1]-ds[i]
    ts2[i] = ts[i+1]-ts[i]
#delete the last data since there is no label for the next day
# print(data)
print(rfr[1:].shape)
print(rfr2.shape)
A = np.stack((rfr[1:], ds[1:], ts[1:], rfr2, ds2, ts2, ones), axis = 1)
# print(A)

#the yield of three major index: Dow Jones, SP 500 and Nasdaq
B_raw = np.stack((data['DJIA'], data['SP500'], data['NASDAQ']), axis = 1)
#Labeled as -1 if the market loss the next day, +1 otherwise 
B = np.zeros((5244, 3))
for i in range(5244):
    for j in range(3):
        B[i][j] = -1 if B_raw[i][j] < 0 else 1

#delete first label since there is no previous day data to predict
B= np.delete(B, 0, 0)

(5243,)
(5243,)


In [19]:
#LASSO, this code is copied from previous assignment
def ista_solve_hot( A, d, la_array ):
    # ista_solve_hot: Iterative soft-thresholding for multiple values of
    # lambda with hot start for each case - the converged value for the previous
    # value of lambda is used as an initial condition for the current lambda.
    # this function solves the minimization problem
    # Minimize |Ax-d|_2^2 + lambda*|x|_1 (Lasso regression)
    # using iterative soft-thresholding.
    max_iter = 10**4
    tol = 10**(-3)
    tau = 1/np.linalg.norm(A,2)**2
    n = A.shape[1]
    w = np.zeros((n,1))
    num_lam = len(la_array)
    X = np.zeros((n, num_lam))
    for i, each_lambda in enumerate(la_array):
        for j in range(max_iter):
            z = w - tau*(A.T@(A@w-d))
            w_old = w
            w = np.sign(z) * np.clip(np.abs(z)-tau*each_lambda/2, 0, np.inf)
            X[:, i:i+1] = w
            if np.linalg.norm(w - w_old) < tol:
                break
    return X


In [20]:
##  10-fold CV 

# each row of setindices denotes the starting an ending index for one
# partition of the data: 5 sets of 30 samples and 5 sets of 29 samples
setindices = [[1,525],[526,1050],[1051,1575],[1576,2100],[2101,2625],[2626,3150],[3151,3675],[3676,4200],[4201,4725],[4726,5243]]

# each row of holdoutindices denotes the partitions that are held out from
# the training set
holdoutindices = [[1,2],[2,3],[3,4],[4,5],[5,6],[7,8],[9,10],[10,1]]

cases = len(holdoutindices)

lam_vals = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
er_sq = np.zeros((cases,1))
er_rate = np.zeros((cases,1))

In [21]:
# DJIA
for j in range(cases):
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    Bv1 = B[v1_ind,:]
   
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    Bv2 = B[v2_ind,:]
    
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    Bt = B[trn_ind,:]
    
    # Use training data to learn classifierA
    W = ista_solve_hot(At,Bt[:,0].reshape((Bt.shape[0],1)),lam_vals)
  
    #calculate error in the 26 w coresponding to each lambda
    er = np.zeros((26, 1))#error of the 26 lambda cases
    miniEr = 1
    miniErIndex = 0
    for i in range(26):
        Bp1 = np.sign(Av1@W[:,i])#predicted b
        error_vec1 = [0 if Bv1[j][0]==Bp1[j] else 1 for j in range(len(Bv1))]
        er[i] = sum(error_vec1) / len(Bv1)
        #select the best lambda with the least error rate
        if miniEr > er[i]:
            miniEr = er[i]
            miniErIndex = i
            
    #get best lambda from above algorithm
    lambda_b = lam_vals[miniErIndex]
    #get best w from the lambda
    w_b = W[:,miniErIndex]
    print(w_b)
    #calculate predicted label for the second holdout evaluation data
    Bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(Bp2 - Bv2[:,0], 2) / len(Bv2[:,0])
    error_vec2 = [0 if Bv2[j][0]==Bp2[j] else 1 for j in range(len(Bv2[:,0]))]
    er_rate[j] = sum(error_vec2) / len(Bv2[:,0])

[ 0.00622335  0.00748424 -0.06298393  0.44854078  1.33264424 -0.06600125
  0.12611149]
[ 0.00352089  0.02139729 -0.05729829  0.06428398  0.17059979 -0.00899731
  0.09803289]
[-0.02327564 -0.01252086 -0.08165596  0.35767141  0.80323955 -0.03928008
  0.19703001]
[-0.02358046 -0.01087595 -0.06713428  0.12538808  1.06111729 -0.00898125
  0.17935599]
[-0.0065878  -0.0191154   0.02623461  0.00815141  0.05398126  0.00113761
  0.0676405 ]
[ 0.00205571  0.00432494  0.00108593  0.01486555  0.02056401 -0.00103783
  0.03904531]
[ 0.00582242  0.02070651 -0.01519199  0.01288277  0.02524496 -0.00232595
  0.01487978]
[-0.00665227 -0.00532629 -0.06465436  0.36127985  0.84489999 -0.06485773
  0.14752798]


In [22]:
print("The classifier based on Dow Jones index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on Dow Jones index:
Square error for each case: 
 [[0.06166253]
 [0.06047432]
 [0.0593846 ]
 [0.06166253]
 [0.0572697 ]
 [0.06011327]
 [0.05842759]
 [0.06095238]]
Average square error: 
 0.059993364752731565
Error rate for each case: 
 [[0.49904762]
 [0.48      ]
 [0.46285714]
 [0.49904762]
 [0.43047619]
 [0.47428571]
 [0.44208494]
 [0.48761905]]
Average error rate: 
 0.47192728442728443


In [23]:
# SP500
for j in range(cases):
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    Bv1 = B[v1_ind,:]
   
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    Bv2 = B[v2_ind,:]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    Bt = B[trn_ind,:]

    # Use training data to learn classifierA
    W = ista_solve_hot(At,Bt[:,1].reshape((Bt.shape[0],1)),lam_vals)
  
    #calculate error in the 26 w coresponding to each lambda
    er = np.zeros((26, 1))#error of the 26 lambda cases
    miniEr = 1
    miniErIndex = 0
    for i in range(26):
        Bp1 = np.sign(Av1@W[:,i])#predicted b
        error_vec1 = [0 if Bv1[j][0]==Bp1[j] else 1 for j in range(len(Bv1))]
        er[i] = sum(error_vec1) / len(Bv1)
        #select the best lambda with the least error rate
        if miniEr > er[i]:
            miniEr = er[i]
            miniErIndex = i
            
    #get best lambda from above algorithm         
    lambda_b = lam_vals[miniErIndex]
    #get best w from the lambda
    w_b = W[:,miniErIndex]
    print(w_b)
    #calculate predicted label for the second holdout evaluation data
    Bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(Bp2 - Bv2[:,1], 2) / len(Bv2[:,1])
    error_vec2 = [0 if Bv2[j][1]==Bp2[j] else 1 for j in range(len(Bv2[:,1]))]
    er_rate[j] = sum(error_vec2) / len(Bv2[:,1])

[ 0.01052032  0.01576843 -0.04056527  0.3919773   1.16270601 -0.05399882
  0.10034759]
[ 7.79965554e-05  1.85161882e-02 -2.08309543e-02  4.32732335e-02
  1.21563353e-01 -6.70790810e-03  7.72016552e-02]
[-0.03217901 -0.0241027  -0.04964507  0.28228847  0.63968615 -0.03037784
  0.19712448]
[-0.02984147 -0.01072604 -0.0798663   0.07627019  0.74562989 -0.00915334
  0.20502757]
[-0.01119385 -0.0156333   0.02959756  0.0051832   0.04826081 -0.0007625
  0.07528695]
[-0.00145163  0.00461927  0.01158064  0.01232175  0.01699851 -0.00105699
  0.03991363]
[ 0.00139437  0.02056817  0.00593299  0.00279186  0.00570833 -0.00056546
  0.01104221]
[-0.01232209 -0.01015915 -0.04721124  0.26606435  0.62321137 -0.05095359
  0.15582105]


In [24]:
print("The classifier based on SP500 index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on SP500 index:
Square error for each case: 
 [[0.06385926]
 [0.05852302]
 [0.06189744]
 [0.06178009]
 [0.05752255]
 [0.06011327]
 [0.05817189]
 [0.06142673]]
Average square error: 
 0.06041177935640187
Error rate for each case: 
 [[0.5352381 ]
 [0.44952381]
 [0.50285714]
 [0.50095238]
 [0.43428571]
 [0.47428571]
 [0.43822394]
 [0.4952381 ]]
Average error rate: 
 0.4788256113256113


In [25]:
# NASDAQ
for j in range(cases):
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    Bv1 = B[v1_ind,:]
   
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    Bv2 = B[v2_ind,:]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    Bt = B[trn_ind,:]

    # Use training data to learn classifierA
    W = ista_solve_hot(At,Bt[:,2].reshape((Bt.shape[0],1)),lam_vals)
    #calculate error in the 26 w coresponding to each lambda
    er = np.zeros((26, 1))#error of the 26 lambda cases
    miniEr = 1
    miniErIndex = 0
    for i in range(26):
        Bp1 = np.sign(Av1@W[:,i])#predicted b
        error_vec1 = [0 if Bv1[j][0]==Bp1[j] else 1 for j in range(len(Bv1))]
        er[i] = sum(error_vec1) / len(Bv1)
        #select the best lambda with the least error rate
        if miniEr > er[i]:
            miniEr = er[i]
            miniErIndex = i
            
    #get best lambda from above algorithm         
    lambda_b = lam_vals[miniErIndex]
    #get best w from the lambda
    w_b = W[:,miniErIndex]
    print(w_b)
    #calculate predicted label for the second holdout evaluation data
    Bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(Bp2 - Bv2[:,2], 2) / len(Bv2[:,2])
    error_vec2 = [0 if Bv2[j][2]==Bp2[j] else 1 for j in range(len(Bv2[:,2]))]
    er_rate[j] = sum(error_vec2) / len(Bv2[:,2])

[-0.00346617 -0.         -0.05003281  0.26203981  0.95644185 -0.0388952
  0.15962068]
[ 0.0083704   0.0389152  -0.04182081  0.03197938  0.09876892 -0.00552093
  0.07675866]
[-0.02169674 -0.00221891 -0.07067127  0.14216232  0.36167036 -0.0155103
  0.19190425]
[-0.02291296 -0.00729486 -0.06895776  0.07368523  0.53326893 -0.00482711
  0.19804243]
[-0.00636845 -0.01202494  0.02623778  0.00862604  0.04786266  0.00019211
  0.08039705]
[ 0.00037164  0.00347786  0.0088996   0.01446395  0.02544302 -0.00125735
  0.05448743]
[ 0.0053073   0.02301317  0.00474952  0.00330289  0.0079611  -0.00061439
  0.01306129]
[-0.01911862 -0.01731654 -0.05345111  0.14087632  0.40974279 -0.02677427
  0.19481236]


In [26]:
print("The classifier based on Nasdaq index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on Nasdaq index:
Square error for each case: 
 [[0.06282828]
 [0.06023386]
 [0.0572697 ]
 [0.06107131]
 [0.0563758 ]
 [0.05839889]
 [0.05829988]
 [0.05975005]]
Average square error: 
 0.05927847172309655
Error rate for each case: 
 [[0.51809524]
 [0.47619048]
 [0.43047619]
 [0.48952381]
 [0.41714286]
 [0.44761905]
 [0.44015444]
 [0.46857143]]
Average error rate: 
 0.460971685971686


In [27]:
#Ridge Regression
#Dow Jones
for j in range(cases):
        # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    bv1 = B[v1_ind,0]
    
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    bv2 = B[v2_ind,0]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    bt = B[trn_ind,0]
    

    Wr = np.zeros((At.shape[1], 26))
    
    miniEr = 1
    miniErIndex = 0
    for i in range(26):
        #train the data with one of the each lambda and add into Wr
        Wr[:,i:i+1] = np.linalg.inv(At.T@At + lam_vals[i]*np.identity(At.shape[1]))@At.T@(bt.reshape(bt.shape[0],1))
        bp1 = np.sign(Av1@Wr[:,i])#predicted b
        error_vec1 = [0 if bv1[j]==bp1[j] else 1 for j in range(len(bv1))]
        er[i] = sum(error_vec1) / len(bv1)
        if miniEr > er[i]:
            miniEr = er[i]
            miniErIndex = i
            
    #get best lambda from above algorithm         
    lambda_b = lam_vals[i]
    #get best w from the lambda
    w_b = Wr[:,i]
    print(w_b)
    #calculate predicted label for the second holdout evaluation data
    bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(bp2 - bv2, 2) / len(bv2)
    error_vec2 = [0 if bv2[j]==bp2[j] else 1 for j in range(len(bv2))]
    er_rate[j] = sum(error_vec2) / len(bv2)


[ 0.00790176  0.00973381 -0.05868183  0.48155665  1.30885961 -0.07038362
  0.11531172]
[-0.00470969  0.0066662  -0.07113158  0.50266709  1.18660526 -0.07216789
  0.14863353]
[-0.02199301 -0.01086906 -0.07587581  0.61194517  1.27117758 -0.07458237
  0.1855465 ]
[-0.02270681 -0.01004891 -0.05640032  0.21047533  1.42649199 -0.01919035
  0.16571154]
[-0.02225957 -0.04439101 -0.06742329  0.20770606  1.10367667  0.0236309
  0.22524929]
[-0.02306692 -0.02840103 -0.05261444  0.56850472  0.8156287  -0.0442079
  0.20390316]
[-7.53446514e-03  7.10282457e-04 -5.53380542e-02  5.92649228e-01
  1.06814628e+00 -1.23992879e-01  1.19473972e-01]
[-0.00554202 -0.003821   -0.06080872  0.59643134  1.25578727 -0.11507859
  0.13882452]


In [28]:
print("The classifier based on Dow Jones index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on Dow Jones index:
Square error for each case: 
 [[0.06178009]
 [0.05962848]
 [0.0593846 ]
 [0.06107131]
 [0.05153428]
 [0.06011327]
 [0.05713764]
 [0.06224813]]
Average square error: 
 0.05911222640582536
Error rate for each case: 
 [[0.50095238]
 [0.46666667]
 [0.46285714]
 [0.48952381]
 [0.34857143]
 [0.47428571]
 [0.42277992]
 [0.50857143]]
Average error rate: 
 0.4592760617760618


In [29]:
#SP500
for j in range(cases):
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    bv1 = B[v1_ind,1]
    
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    bv2 = B[v2_ind,1]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    bt = B[trn_ind,1]
    
    Wr = np.zeros((At.shape[1], 26))
    
    miniEr = 1
    miniErIndex = 0
    for i in range(26):
        #train the data with one of the each lambda and add into Wr
        Wr[:,i:i+1] = np.linalg.inv(At.T@At + lam_vals[i]*np.identity(At.shape[1]))@At.T@(bt.reshape(bt.shape[0],1))
        bp1 = np.sign(Av1@Wr[:,i])#predicted b
        error_vec1 = [0 if bv1[j]==bp1[j] else 1 for j in range(len(bv1))]
        er[i] = sum(error_vec1) / len(bv1)
        if miniEr > er[i]:
            miniEr = er[i]
            miniErIndex = i
            
    #get best lambda from above algorithm         
    lambda_b = lam_vals[i]
    #get best w from the lambda
    w_b = Wr[:,i]
    print(w_b)
    #calculate predicted label for the second holdout evaluation data
    bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(bp2 - bv2, 2) / len(bv2)
    error_vec2 = [0 if bv2[j]==bp2[j] else 1 for j in range(len(bv2))]
    er_rate[j] = sum(error_vec2) / len(bv2)

[ 0.01155549  0.01708646 -0.03758303  0.46902096  1.25797219 -0.06378074
  0.09342229]
[-0.00955752  0.00151687 -0.03867826  0.47005158  1.1579174  -0.07468433
  0.1379266 ]
[-0.03121271 -0.02286926 -0.04525434  0.5876414   1.22190397 -0.07055756
  0.18834834]
[-0.02971304 -0.01071471 -0.07684232  0.17828151  1.33440053 -0.02418504
  0.20183877]
[-0.02849359 -0.0434462  -0.07524961  0.13961716  1.00119199 -0.02382292
  0.2505138 ]
[-0.02818793 -0.03185923 -0.03673293  0.50863434  0.72898398 -0.05006385
  0.20836744]
[-0.01978227 -0.01336213 -0.04123317  0.55072076  1.02392056 -0.12929662
  0.16326784]
[-0.01177139 -0.00956077 -0.04463617  0.57040923  1.18790979 -0.11825609
  0.15101682]


In [30]:
print("The classifier based on SP500 index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on SP500 index:
Square error for each case: 
 [[0.06363159]
 [0.05814986]
 [0.06154474]
 [0.06119001]
 [0.05153428]
 [0.06011327]
 [0.05791506]
 [0.06305884]]
Average square error: 
 0.059642206949073365
Error rate for each case: 
 [[0.53142857]
 [0.44380952]
 [0.49714286]
 [0.49142857]
 [0.34857143]
 [0.47428571]
 [0.43436293]
 [0.52190476]]
Average error rate: 
 0.46786679536679543


In [31]:
#NASDAQ
for j in range(cases):
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    bv1 = B[v1_ind,2]
    
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    bv2 = B[v2_ind,2]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    bt = B[trn_ind,2]
    
    Wr = np.zeros((At.shape[1], 26))
    
    miniEr = 1
    miniErIndex = 0
    for i in range(26):
        #train the data with one of the each lambda and add into Wr
        Wr[:,i:i+1] = np.linalg.inv(At.T@At + lam_vals[i]*np.identity(At.shape[1]))@At.T@(bt.reshape(bt.shape[0],1))
        bp1 = np.sign(Av1@Wr[:,i])#predicted b
        error_vec1 = [0 if bv1[j]==bp1[j] else 1 for j in range(len(bv1))]
        er[i] = sum(error_vec1) / len(bv1)
        if miniEr > er[i]:
            miniEr = er[i]
            miniErIndex = i
            
    #get best lambda from above algorithm         
    lambda_b = lam_vals[i]
    #get best w from the lambda
    w_b = Wr[:,i]
    print(w_b)
    #calculate predicted label for the second holdout evaluation data
    bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(bp2 - bv2, 2) / len(bv2)
    error_vec2 = [0 if bv2[j]==bp2[j] else 1 for j in range(len(bv2))]
    er_rate[j] = sum(error_vec2) / len(bv2)

[-0.00258208  0.00156134 -0.04740868  0.38442053  1.20617328 -0.05821719
  0.15224698]
[-0.00313069  0.01904492 -0.06580196  0.41608266  1.10658681 -0.07189426
  0.15161355]
[-0.02290634 -0.00473378 -0.07081173  0.50605179  1.14441854 -0.06089941
  0.19790121]
[-0.02421108 -0.00875162 -0.08019888  0.22217647  1.25698172 -0.01803773
  0.21352399]
[-0.0241653  -0.03978544 -0.09208329  0.21034531  0.96874787  0.00157928
  0.26925805]
[-0.02604991 -0.03162634 -0.04422114  0.4503785   0.78735573 -0.04262816
  0.22521648]
[-0.01323917 -0.00584977 -0.04392421  0.48695245  1.0315244  -0.10108605
  0.15260382]
[-0.02054365 -0.01993565 -0.05449986  0.4627919   1.13944959 -0.09244563
  0.20283843]


In [33]:
print("The classifier based on NASDAQ index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on NASDAQ index:
Square error for each case: 
 [[0.06248084]
 [0.06023386]
 [0.05877047]
 [0.06059419]
 [0.05096795]
 [0.05839889]
 [0.05791506]
 [0.06083322]]
Average square error: 
 0.058774309576769795
Error rate for each case: 
 [[0.51238095]
 [0.47619048]
 [0.45333333]
 [0.48190476]
 [0.34095238]
 [0.44761905]
 [0.43436293]
 [0.48571429]]
Average error rate: 
 0.45405727155727155
